In [13]:
import pandas as pd
import numpy as np
import sys
import os
import scipy.stats as stats
from statistics import mean
from enum import Enum
import sys
print(sys.version)
import math

class DB(Enum):
    Mcyt=0
    eBioSignDS1=1
    eBioSignDS2=3
    BiosecurID=4
    BiosecureDS2=5
    EvalDB=6

class Signature:
    def __init__(self, data, filePath):
        self.data = data
        self.filePath = filePath
        pathParts= filePath.split('/')
        self.inputDevice=pathParts[2]
        if "signature" in filePath:
            self.db='EvalDb'
            self.inputDevice='Unknown'
            return
        
        parts=pathParts[4].split('_')
        signerId=parts[0][1:]
        self.db=self.getDatabase(pathParts[2],pathParts[3],int(signerId))
        #print(self.db)
        pressureColumn=self.getPressureColumn()
        
        self.x=data[:, 0].tolist()
        self.y=data[:, 1].tolist()
        self.t=data[:, 2].tolist()
        self.pressure=data[:, pressureColumn]
        
        if (self.inputDevice == "Unkown"):
            self.inputDevice=getInputDeviceForEvalDb()
        
        self.correctZeroTime()
        self.correctSameTimestamps()
    
        if self.inputDevice == 'stylus':
            i=0
            while len(self.pressure) > 0 and self.pressure[0] == 0:
                self.x.pop(0)
                self.y.pop(0)
                self.t.pop(0)
                self.pressure.pop(0)

            while (len(self.pressure) > 0 and self.pressure[len(self.pressure)-1] == 0):
                self.x.pop(len(self.pressure)-1)
                self.y.pop(len(self.pressure)-1)
                self.t.pop(len(self.pressure)-1)
                self.pressure.pop(len(self.pressure)-1)
        
        self.dx=difference(self.x)
        self.dy=difference(self.y)
        
        self.calculatePathTangentAngle()
        self.calculatePVM()
        self.dpvm=difference(self.pvm)
        self.dpta=difference(self.pta)
        self.lcr=self.calculateLCR()

        self.calculateTotalAccelerationMagnitude()
        self.dtam=difference(self.tam)
        #self.pta=math.atan(self.dy/self.dx)
        #self.pvm=math.sqrt(self.dx**2+self.dy*2)
        #self.dx=self.x.diff()
        #self.lcr=math.log(self.pvm/self.pta,10)

        self.x=normalize(self.x)
        self.y=normalize(self.y)
    
        if self.inputDevice != "finger":
            self.dpressure=difference(self.pressure)
            self.dpressure=normalize(self.dpressure)    
            self.pressure=normalize(self.pressure)
        
        self.pvm=normalize(self.pvm)
        self.pta=normalize(self.pta)
        self.tam=normalize(self.tam)
        self.dpvm=normalize(self.dpvm)
        self.dpta=normalize(self.dpta)
        
        self.ddx=difference(self.dx)
        self.ddy=difference(self.dy)
        
        '''
        print('self.x') 
        print(self.x) 
        print(self.filePath) 
        print(self.dx)  
        print(self.dy)
        print(self.dpressure)
        print(self.pta)
        print(self.pvm)
        #print(self.lcr)
        print(self.tam)
        '''
        
    
    def calculatePathTangentAngle(self):
        self.pta=[]
        for i in range(0,len(self.dx)):
            if(self.dx[i]==0):
                self.pta.append(math.atan(self.dy[i]/(self.dx[i] + 0.0000000000000001)))
            else:
                self.pta.append(math.atan(self.dy[i]/self.dx[i]))
        
    def calculatePVM(self):
        self.pvm=[]
        for i in range(0,len(self.dx)):
            self.pvm.append(math.sqrt(self.dx[i]**2+self.dy[i]**2))
            
    def calculateLCR(self):
        self.lcr=[]
        for i in range(0,len(self.dpta)):
            value=0
            if(self.dpta[i]==0):
                value=self.pvm[i]/(self.dpta[i]+0.0000000000000001)
            else:
                value=self.pvm[i]/self.dpta[i]
            value=abs(value)
            if(value<=0):
                print("value<=0 at index:")
                print(i)
                print('value:')
                print(value)
                print('self.dx')
                print(self.dx)
                print('self.dy') 
                print(self.dy)
                print('self.pta') 
                print(self.pta)
                print('self.dpta') 
                print(self.dpta)
                print('self.pvm')
                print(self.pvm)
                
                self.lcr.append(math.log(value+0.0000000000000001,10))
            else:
                self.lcr.append(math.log(value,10))
    
    def calculateTotalAccelerationMagnitude(self):
        self.tam=[]
        for i in range(0,len(self.dpvm)):
            self.pta.append(math.sqrt(self.dpvm[i]**2+(self.pvm[i]**2)*(self.pta[i]**2)))
            
    def getPressureColumn(self):
        if self.db==DB.Mcyt:
            return 5
        elif self.db==DB.BiosecurID:
            return 6
        elif self.db==DB.BiosecureDS2:
            return 6
        elif self.db==DB.eBioSignDS1:
            return 3
        elif self.db==DB.eBioSignDS2:
            return 3
        elif self.db==DB.EvalDB:
            return 3
        
    def getDatabase(self, split, inputDevice, signerId):
        '''
        print(split)
        print(inputDevice)
        print(signerId)
        '''
        if split=='Development':
            if inputDevice=='finger':
                if 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=230: 
                    return DB.Mcyt
                elif 231<=signerId<=498:
                    return DB.BiosecurID
                elif 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        elif split=='Evaluation':
            if inputDevice=='finger':
                if 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=100:
                    return DB.Mcyt
                elif 101<=signerId<=232:
                    return DB.BiosecurID
                elif 233<=signerId<=372:
                    return DB.BiosecureDS2
                elif 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        else:
            print("Undefined InputDevice for file: {file}")
            
    def getInputDeviceForEvalDb(self):
        for i in range(0, len(self.pressure), 1):
            if self.pressure[i]!=0:
                return False

        return True
    
    def isAllZero(self):
        for i in range(0, self.data.shape[0], 1):
            if self.data[i,0]!=0:
                return False

        return True
    
    def correctZeroTime(self):
        if self.isAllZero():
            for i in range(0, self.data.shape[0], 1):
                self.data[i,0]=i*10
                
    def correctSameTimestamps(self):
        i=0
        while i < self.data.shape[0]-1:
            if self.data[i,0]==self.data[i+1,0]:
                self.data=np.delete(self.data, i, 0)
            else:
                i += 1

def readFilesInDir(dirPath, allFiles):
    signatureFiles = os.listdir(dirPath)
    for file in signatureFiles:
        dataframe=pd.read_csv(dirPath + '/' + file,sep=' ',skiprows=[0],header=None)
        array=np.array(dataframe)
        signature=Signature(array,dirPath + '/' + file)
        allFiles.append(signature)
        
def difference(dataset, interval=1):
    diff = []
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
        
    return diff

def Average(lst):
    return sum(lst) / len(lst)

def normalizeX(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.x
    
    #print('xAll')
    #print(xAll)
    #average=mean(xAll)
    z = stats.zscore(all)
    #print(average)
    print(z)
    allIndex=0
    
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized x:')
    for item in allFiles:
        print(item.x)
    '''
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=(item.x[i]-average)/z
    '''
def normalizeY(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.y
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.y),1):
            item.y[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized y:')
    for item in allFiles:
        print(item.y)
            
def normalizePressure(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.pressure
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized pressure:')
    for item in allFiles:
        print(item.filePath)
        print(item.pressure)

def normalize(data):
    normalized=[]
    z = stats.zscore(data)
    #print(z)

    for i in range(0,len(data),1):
        normalized.append(z[i])
            
    return normalized
'''
for item in normalized:
        print(item)
        
    return normalized
'''
    
        
def normalizeY2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.y)
        print(z)

        for i in range(0,len(item.y),1):
            item.y[i]=z[i]

    print('Normalized y:')
    for item in allFiles:
        print(item.y)
        
def normalizePressure2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.pressure)
        print('item.pressure:')
        print(item.pressure)
        print('z:')
        print(z)

        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[i]

    print('Normalized pressure:')
    for item in allFiles:
        print("item.filePath")
        print(item.pressure)

def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

def dtwAllX(allFiles):
    '''
    for i in range(0,len(allFiles)):
        for j in range(1,)
    '''
    return dtw(allFiles[0].x,allFiles[1].x)
            
allFiles=[]
        
#readFilesInDir('C:\DeepSignDB/Evaluation/stylus',allFiles)
#readFilesInDir('C:\DeepSignDB/Evaluation/finger',allFiles)
#readFilesInDir('C:\DeepSignDB/Development/stylus',allFiles)

readFilesInDir('D:/DeepSignSmall/Evaluation/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Evaluation/finger',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/finger',allFiles)

'''
for file in allFiles:
    #print(file.data)
    #print(file.filePath)
    #print(file.db)
    #print(file.pressure)
'''
    
print('Allfiles:')
#print(allFiles)
print('Loading data finished')
dtwMatrix=dtwAllX(allFiles)
print(dtwMatrix)

#normalizeX2(allFiles)
#normalizeY2(allFiles)
#normalizePressure2(allFiles)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
value<=0 at index:
20
value:
0.0
self.dx
[113, -23, -66, -95, -122, -193, -241, -192, -126, -81, 0, 50, 81, 147, 180, 203, 204, 149, 112, 30, 0, -17, -76, -124, -155, -112, 5, 84, 142, 187, 217, 185, 121, 78, 48, -29, -81, -153, -212, -252, -270, -145, -62, 157, 262, 331, 299, 227, 180, 143, 49, -13, -106, -217, -291, -362, -398, -421, -158, 55, 197, 356, 407, 440, 352, 267, 209, 43, -21, -63, -155, -281, -364, -398, -362, -338, -28, 182, 321, 423, 364, 324, 181, 37, -58, -183, -337, -439, -520, -482, -457, -200, -44, 60, 119]
self.dy
[98, 0, -34, -98, -179, -233, -291, -254, -229, -87, -12, 38, 127, 195, 241, 291, 277, 267, 105, 41, 0, -18, -120, -189, -256, -252, -250, -116, -36, 17, 100, 125, 143, 216, 211, 138, 92, 23, -69, -129, -198, -205, -209, -134, -50, 6, 75, 91, 101, 145, 111, 89, 54, 0, -38, -123, -190, -234, -148, -118, -98, 0, 59, 95, 109, 115, 118, 50, 37, 29, 17, -41, -80, -127, -137, -144, -42, -10, 12,

0.0
self.dx
[-69, -30, -18, 0, 11, 33, 52, 65, 66, 49, 21, -11, -57, -111, -163, -211, -236, -230, -195, -129, -53, 40, 138, 234, 317, 375, 400, 385, 354, 299, 235, 169, 105, 0, 47, -43, -82, -117, -149, -172, -191, -197, -194, -192, -179, -161, -143, -110, -74, -43, -11, 6, 10, 0, 0, 0, -73, -56, -67, -72, -65, -46, -22, 0, 14, 32, 47, 56, 58, 45, 27, 12, 0, -7, -10, -10, -9, 0, -5, 0, 5, 10, 15, 23, 32, 35, 36, 36, 29, 25, 18, 8, 0, 0, -8, -17, -24, -20, -13, 0, 18, 39, 61, 84, 98, 102, 101, 93, 71, 43, 12, -17, -41, -61, -72, -75, -61, -38, -15, 9, 25, 36, 49, 58, 54, 48, 30, 9, -5, -22, -30, -32, -34, -18, -5, 7, 25, 35, 39, 38, 26, 11, -6, -25, -35, -39, -34, 0, -7, 30, 56, 76, 79, 72, 47, 0, 0, 0, 0, 0, -42, 0, 0, 12, 12, 6, 0, -25, -31, -38, -40, -30, 0, -8, 35, 63, 94, 116, 130, 137, 125, 104, 75, 45, 0, -4, -32, -52, -56, -58, -47, -24, 0, 26, 45, 62, 68, 63, 44, 21, 0, -29, -42, -54, -65, -65, -64, -60, -48, -35, -24, -8, 0, 31, 39, 55, 68, 79, 83, 73, 53, 0, 23, -6, -20, -17

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




self.pvm
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.656854249492381, 21.213203435596427, 29.68164415931166, 20.615528128088304, 37.53664875824692, 1.0, 9.0, 3.605551275463989, 5.385164807134504, 8.94427190999916, 6.708203932499369, 3.605551275463989, 5.385164807134504, 7.211102550927978, 7.0710678118654755, 4.242640687119285, 1.4142135623730951, 0.0, 0.0, 1.0, 0.0, 1.4142135623730951, 7.211102550927978, 8.48528137423857, 52.40229002629561, 15.0, 10.816653826391969, 23.323807579381203, 0.0, 9.219544457292887, 16.64331697709324, 5.0, 16.1245154965971, 15.620499351813308, 1.4142135623730951, 2.23606797749979, 3.605551275463989, 2.23606797749979, 3.605551275463989, 2.23606797749979, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 4.123105625617661, 0.0, 4.0, 3.0, 26.627053911388696, 19.1049731745428, 5.0, 5.385164807134504, 8.06225774829855, 6.708203932499369, 5.0, 4.242640687119285, 4.242640687119285, 4.123105625617661, 3.1622776601683795, 2.0, 2.23606797749979, 1.41421356237

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




value:
0.0
self.dx
[0, 4, 1, 2, 2, 3, 5, 14, 10, 11, 6, 4, 5, 7, 7, 17, 5, 4, 4, 4, 1, 1, 2, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, -1, -2, 0, 0, 0, 0, 0, 3, 2, 1, 3, 4, 4, 4, 3, 3, 2, 2, 1, 1, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -4, -2, -3, -2, -3, -1, -2, -1, -1, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -1, -2, -1, -1, -1, -2, -2, -2, -3, -5, -6, -9, -6, -6, -3, -4, -3, -4, -3, -4, -4, -2, -1, 0, 0, 4, 2, 3, 3, 4, 5, 8, 9, 9, 10, 12, 25, 22, 27, 15, 7, 5, 3, 2, 1, 0, 0, -3, -3, -3, -4, -3, -5, -5, -4, -3, -2, -3, 0, 2, 4, 7, 6, 4, 3, 3, 4, 4, 4, 4, 5, 7, 6, 6, 7, 4, 2, 1, 1, 1, 0, 0, 0, 0, -3, 0, -1, 0, 0, 5, 2, 3, 4, 5, 8, 5, 4, 3, 3, 3, 2, 2, 1, 1, 0, 0, -1, -2, -2, -1, -4, -1, -1, 0, 4, 3, 3, 3, 5, 6, 8, 9, 6, 4, 3, 1, 1, 0, -4, -4, -5, -5, -4, -3, -2, -2, 1, 0, 5, 5, 5, 4, 4, 3, 4, 3, 3, 5, 4, 7, 15, 9, 6, 6, 5, 5, 3, 4, 2, 1, 1, -4, -4, -3, -3, -4, -5, -6, -8, -10, -6, -2, -1, 0, 5, 6, 6, 4, 4, 4, 4, 3, 3, 2, 3, 2, 0, 0, 0, 0, 0, -2, 0, -2, 0, 0, -1, -2, 0, -1, -1, 0, 0